In [7]:
import pandas as pd
import numpy as np
import snoop
from pprint import pprint as print

# %load_ext snoop
import xgboost as xgb
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


In [2]:
df = pd.read_csv("transactions by month.csv")
print(df)


In [8]:
def foo(x):
    if '2018' in x:
        return '2018'
    elif '2019' in x:
        return '2019'
    else:
        return 'other'


df['year'] = df['month'].apply(lambda x: foo(x))
df['margin'] = df['payment'] - df['primecost']
df


,month,action,is_partner,servtype,users,services,items,payment,primecost,year,margin
0,2018-01,new,0,domain,6107,9811,9813,2273837,1852603,2018,421234
1,2018-01,new,1,domain,1118,6702,6702,1301999,1150975,2018,151024
2,2018-01,renew,0,domain,5879,7323,7324,6176681,1499403,2018,4677278
3,2018-01,renew,1,domain,3315,11978,11984,2852070,1881500,2018,970570
4,2018-02,new,0,domain,6468,10445,10445,2383773,1918312,2018,465461
...,...,...,...,...,...,...,...,...,...,...,...
280,2019-08,renew,1,hosting,326,468,471,254977,120554,2019,134423
281,2019-09,renew,1,hosting,338,451,453,268282,127668,2019,140614
282,2019-10,renew,1,hosting,368,482,483,292319,138769,2019,153550
283,2019-11,renew,1,hosting,328,461,464,255781,121364,2019,134417


In [4]:
t = df.groupby('month').sum().sort_index()
t.rolling(3).mean()

,is_partner,users,services,items,payment,primecost,margin
month,,,,,,,
2018-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03,5.666667,21756.666667,41189.000000,41235.666667,2.130990e+07,8.556525e+06,1.275337e+07
2018-04,5.666667,21489.000000,40331.000000,40373.333333,2.156628e+07,8.491106e+06,1.307518e+07
2018-05,5.666667,21299.666667,40285.000000,40330.666667,2.120063e+07,8.577167e+06,1.262346e+07
2018-06,6.000000,20154.666667,38443.666667,38487.333333,2.024654e+07,8.190286e+06,1.205626e+07
2018-07,6.000000,19659.333333,38323.666667,38367.666667,1.970542e+07,8.198915e+06,1.150650e+07
2018-08,6.000000,19623.333333,37902.666667,37948.000000,1.985659e+07,8.163963e+06,1.169263e+07
2018-09,5.666667,20261.666667,38557.000000,38600.666667,2.042310e+07,8.471048e+06,1.195205e+07


In [5]:
t = df.groupby(["is_partner", "year", "servtype"]).sum()[
    ["payment", "primecost", "services", "items"]
]
t
new = 1

payment  primecost  services   items
is_partner year servtype                                        
0          2018 domain    103577937   44754036    211661  211715
                hosting    48494077   23034693     62284   62613
                server     51350617    1142258      5172    5258
           2019 domain    120498613   49470216    235026  235123
                hosting    63720960   30269421     81726   82156
                server     62333258     610751      5114    5175
1          2018 domain     44984403   32944853    195829  195887
                hosting     3378205    1604667      5338    5385
                server      2301385      64627       240     240
           2019 domain     48064161   34495303    206795  206845
                hosting     3847435    1824699      6090    6129
                server      2482046      56830       282     284

In [6]:
t['payment']-t['primecost']

is_partner  year  servtype
0           2018  domain      58823901
                  hosting     25459384
                  server      50208359
            2019  domain      71028397
                  hosting     33451539
                  server      61722507
1           2018  domain      12039550
                  hosting      1773538
                  server       2236758
            2019  domain      13568858
                  hosting      2022736
                  server       2425216
dtype: int64